In [ ]:
import requests
import json
import time
import re

# Initial Setup -----------------------------------------------------------

# Set StreetLight key and email
StlKey = "DgO7zCdYednFilSOR1dTX7jsmsdcoz31"
email = "chris.ryan@hdrinc.com"

# tmap_mode("view")
proj = 6595

# Load data
import geopandas as gpd

allGrid, stationArea, stationPoints = gpd.read_file("shape/stationArea.geojson"), gpd.read_file("shape/RailStationAreas.shp").to_crs(epsg=proj), gpd.read_file("shape/stationPoints.shp").to_crs(epsg=proj)

# Define StreetLight API Call Functions ------------------------------------

# Preset variables for testing functions
mode = "Pedestrian"
zoneset = "StationAreas"

def StL_API_OD(mode, zoneset):
    analysisName = f"{zoneset}_{mode}_OD"

    bodyA = {
        "insight_login_email": email,
        "analysis_name": analysisName,
        "analysis_type": "OD_Analysis",
        "travel_mode_type": mode,  # All_Vehicles/Truck/Bicycle/Pedestrian/Bus/Rail
        "enable_visualization": True,
        "oz_sets": [{"name": zoneset}],
        "dz_sets": [{"name": zoneset}],
        "date_ranges": [{
            "start_date": "01/01/2019",
            "end_date": "12/31/2019"
        }],
        "day_types": "Mon|11, Tue|22, Wed|33, Thu|44, Fri|55, Sat|66, Sun|77",
        "day_parts": "All Day|0023,12am|0000,1am|0101,2am|0202,3am|0303,4am|0404,5am|0505,6am|0606,7am|0707,8am|0808,9am|0909,10am|1010,11am|1111,12pm|1212,1pm|1313,2pm|1414,3pm|1515,4pm|1616,5pm|1717,6pm|1818,7pm|1919,8pm|2020,9pm|2121,10pm|2222,11pm|2323",
        "output_type": "index"
        # "segment_types": ["Motorway", "Trunk", "Primary", "Secondary", "Tertiary"]  # only need for top route
    }

    bodyAJSON = json.dumps(bodyA)

    # Call to create Analysis
    url = f"https://insight.streetlightdata.com/api/v2/analyses?key={StlKey}"
    api_call("POST", url, bodyAJSON)
    time.sleep(6)  # Max analysis creation is 10 per minute
    print(analysisName)


def StL_API_PresetOD(mode, zoneset):
    analysisName = f"{zoneset}_{mode}_PresetOD"

    bodyA = {
        "insight_login_email": email,
        "analysis_name": analysisName,
        "analysis_type": "OD_Preset_Geography",
        "travel_mode_type": mode,  # All_Vehicles/Truck/Bicycle/Pedestrian/Bus/Rail
        "enable_visualization": True,
        "oz_sets": [{"name": zoneset}],
        # "dz_sets": [{"name": zoneset}],
        "geography_type": "blkgrp",
        "date_ranges": [{
            "start_date": "01/01/2019",
            "end_date": "12/31/2019"
        }],
        "day_types": "Mon|11, Tue|22, Wed|33, Thu|44, Fri|55, Sat|66, Sun|77",
        "day_parts": "All Day|0023,12am|0000,1am|0101,2am|0202,3am|0303,4am|0404,5am|0505,6am|0606,7am|0707,8am|0808,9am|0909,10am|1010,11am|1111,12pm|1212,1pm|1313,2pm|1414,3pm|1515,4pm|1616,5pm|1717,6pm|1818,7pm|1919,8pm|2020,9pm|2121,10pm|2222,11pm|2323",
        "output_type": "index"
    }

    bodyAJSON = json.dumps(bodyA)

    # Call to create Analysis
    url = f"https://insight.streetlightdata.com/api/v2/analyses?key={StlKey}"
    api_call("POST", url, bodyAJSON)
    time.sleep(6)  # Max analysis creation is 10 per minute
    print(analysisName)


def StL_API_TopRoute(mode, zoneset):
    analysisName = f"{zoneset}_{mode}_TopRoute"

    bodyA = {
        "insight_login_email": email,
        "analysis_name": analysisName,
        "analysis_type": "Top_Routes_ZA",
        "travel_mode_type": mode,  # All_Vehicles/Truck/Bicycle/Pedestrian/Bus/Rail
        "enable_visualization": True,
        "oz_sets": [{"name": zoneset}],
        "dz_sets": [{"name": zoneset}],
        "geography_type": "blkgrp",
        "date_ranges": [{
            "start_date": "01/01/2019",
            "end_date": "12/31/2019"
        }],
        "day_types": "Mon|11, Tue|22, Wed|33, Thu|44, Fri|55, Sat|66, Sun|77",
        "day_parts": "All Day|0023,12am|0000,1am|0101,2am|0202,3am|0303,4am|0404,5am|0505,6am|0606,7am|0707,8am|0808,9am|0909,10am|1010,11am|1111,12pm|1212,1pm|1313,2pm|1414,3pm|1515,4pm|1616,5pm|1717,6pm|1818,7pm|1919,8pm|2020,9pm|2121,10pm|2222,11pm|2323",
        "output_type": "index",
        "segment_types": ["Motorway", "Trunk", "Primary", "Secondary", "Tertiary"]  # only need for top route
    }

    bodyAJSON = json.dumps(bodyA)

    # Call to create Analysis
    url = f"https://insight.streetlightdata.com/api/v2/analyses?key={StlKey}"
    api_call("POST", url, bodyAJSON)
    time.sleep(6)  # Max analysis creation is 10 per minute
    print(analysisName)


# Create Analyses -------------------------------------------------

# Loop to create Ped/Bike OD for Hex Grids
for i, row in stationArea.iterrows():
    zoneset = f"{stationPoints.iloc[i]['name']}1"
    StL_API_OD("Pedestrian", zoneset)
    StL_API_OD("Bicycle", zoneset)
    print(zoneset)

# API call for Top Route and Preset OD
StL_API_PresetOD("All_Vehicles", "StationAreas2021-09-17")
StL_API_TopRoute("All_Vehicles", "StationAreas2021-09-17")


# Handling Forward Slash Name Issues --------------------------------------

# Redo function to be able to have separately defined zoneset and analysis name
def StL_API_ODv2(mode, zoneset, analysisName):
    analysisName = f"{analysisName}_{mode}_OD"

    bodyA = {
        "insight_login_email": email,
        "analysis_name": analysisName,
        "analysis_type": "OD_Analysis",
        "travel_mode_type": mode,  # All_Vehicles/Truck/Bicycle/Pedestrian/Bus/Rail
        "enable_visualization": True,
        "oz_sets": [{"name": zoneset}],
        "dz_sets": [{"name": zoneset}],
        "date_ranges": [{
            "start_date": "01/01/2019",
            "end_date": "12/31/2019"
        }],
        "day_types": "Mon|11, Tue|22, Wed|33, Thu|44, Fri|55, Sat|66, Sun|77",
        "day_parts": "All Day|0023,12am|0000,1am|0101,2am|0202,3am|0303,4am|0404,5am|0505,6am|0606,7am|0707,8am|0808,9am|0909,10am|1010,11am|1111,12pm|1212,1pm|1313,2pm|1414,3pm|1515,4pm|1616,5pm|1717,6pm|1818,7pm|1919,8pm|2020,9pm|2121,10pm|2222,11pm|2323",
        "output_type": "index"
        # "segment_types": ["Motorway", "Trunk", "Primary", "Secondary", "Tertiary"]  # only need for top route
    }

    bodyAJSON = json.dumps(bodyA)

    # Call to create Analysis
    url = f"https://insight.streetlightdata.com/api/v2/analyses?key={StlKey}"
    api_call("POST", url, bodyAJSON)
    time.sleep(6)  # Max analysis creation is 10 per minute
    print(analysisName)

# Rerun analyses to handle forward slash in Broad Run and Franconia
extra = stationPoints[stationPoints['name'].str.contains("/", regex=False)].copy()
extra['aName'] = extra['name'].str.replace("/", "-")

for i, row in extra.iterrows():
    analysisName = f"{row['aName']}1"
    zoneset = f"{row['name']}1"
    StL_API_ODv2("Pedestrian", zoneset, analysisName)
    StL_API_ODv2("Bicycle", zoneset, analysisName)
    print(zoneset)

# Rerunning analyses for Norfolk (First try was using incorrect zone set)
StL_API_ODv2("Pedestrian", "NorfolkStation1 - Copy", "Norfolk1")
StL_API_ODv2("Bicycle", "NorfolkStation1 - Copy", "Norfolk1")
